PoolFormer
====


**MetaFormer Is Actually What You Need for Vision**

 * Paper: https://arxiv.org/abs/2111.11418

![PoolFormer Architecture](../assets/poolformer_architecture.png)

![PoolFormer Comparison](../assets/poolformer_comparison.png)